<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/04_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 単語ベクトル


## 今日のお題
* 単語ベクトルを利用して、テキストをベクトル化する。
* ベクトル化を埋め込み(embedding)と呼ぶ。
  * 以下、埋め込みという言い方を使う。
* こうして作ったベクトルを使って、テキスト分類問題を解く。
* 同じ分類問題を、BERTでテキストをembedすることによって解く。
* 両者の性能を比較する。

## 単語ベクトルとは
* いわゆるword2vec。
  * https://arxiv.org/abs/1301.3781
  * https://en.wikipedia.org/wiki/Word2vec
* 単語をベクトルとして表現したもの。
  * 単語埋め込み、単語分散表現、などとも言われる。
* 意味が近い単語はベクトルとしても近くなるように、作成されている。


## 単語ベクトルを作るアルゴリズム
* アルゴリズム自体の説明は、この授業では割愛します。
  * https://www.tensorflow.org/text/tutorials/word2vec
* 大雑把には・・・
  * テキストをたくさん集める。それらのテキストの中で・・・
  * 各単語について、前後にどのような単語が出現するか、調べる。
  * 前後に似たような単語が出現する単語は、似たようなベクトルにマッピングする。

## 単語ベクトルの使いみち

### 単語の類似度評価
* ベクトルどうしの遠い近いを表す尺度は何でも使える。
* 内積やコサイン類似度が使われることが多い。


### テキスト埋め込み
* 最もシンプルには、テキストに含まれるトークンの単語ベクトルの平均を取ればよい。
  * これをmean poolingと呼ぶ。
* 単語ベクトルを使ってテキストを埋め込むことは、最近は行わない。
  * テキストのembedには、今は、深層学習言語モデルを使う。
  * 今回はsentence BERTを使った方法を紹介する。

## 準備

### 環境設定
* 今回はランタイムのタイプでGPUを選んでおいてください。
  * あとでBERTによるテキスト埋め込みと比較するため。

### 必要なライブラリのインストール

In [ ]:
!pip install datasets

## データセット

### WRIME: 主観と客観の感情分析データセット
* 詳細は、以下を参照。
  * https://github.com/ids-cv/wrime
* 短いテキストがたくさん含まれている。
* -2, -1, 0, 1, 2の５段階でnegativeからpositiveの感情ラベルが付与されている。
* 今回は、Hugging Face Hubからこのデータセットを取得する。


In [ ]:
from datasets import load_dataset

dataset = load_dataset("shunk031/wrime", "ver2")

* はじめから、train, validation, testの3つの集合に分けられている。

In [ ]:
import numpy as np

tags = ["train", "validation", "test"]

texts = {}
labels = {}
for tag in tags:
  texts[tag] = dataset[tag]["sentence"]
  labels[tag] = [item["sentiment"] for item in dataset[tag]["avg_readers"]]
  labels[tag] = np.array(labels[tag])

In [ ]:
texts["train"][0]

In [ ]:
labels["train"][0]

## 単語ベクトルによるテキストの埋め込み
* 小規模のモデル（名前が__`_sm`__で終わるモデル）は単語ベクトルを含まない。
* 大規模モデルはダウンロードに時間がかかる。
* そのため、中規模モデルをインストールする。

### 日本語中規模モデルのインストール
* https://spacy.io/models/ja#ja_core_news_md

In [ ]:
!python -m spacy download ja_core_news_md

### テキストの埋め込み
* spaCyではテキストを直接embedできる。
  * 内部では単語ベクトルの平均を求めている。
* （おそらく6分ぐらいかかります。）

In [ ]:
from tqdm import tqdm
import numpy as np
import spacy

nlp = spacy.load('ja_core_news_md')

X = {}
for tag in tags:
  X[tag] = []
  for text in tqdm(texts[tag]):
    tokens = nlp(text)
    X[tag].append(tokens.vector)
  X[tag] = np.array(X[tag])

In [ ]:
X["train"].shape

* embedした結果とラベルを保存しておく。

In [ ]:
for tag in tags:
  with open(f'wrime_{tag}_vec.npy', 'wb') as f:
    np.save(f, X[tag])
  with open(f'wrime_{tag}_label.npy', 'wb') as f:
    np.save(f, labels[tag])

### ラベルの前処理

* 保存しておいたテキストのベクトル表現とラベルを読み込む。

In [ ]:
import numpy as np

tags = ["train", "validation", "test"]

X = {}
labels = {}
for tag in tags:
  with open(f'wrime_{tag}_vec.npy', 'rb') as f:
    X[tag] = np.load(f)
  with open(f'wrime_{tag}_label.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
X["train"].shape

In [ ]:
labels["train"].shape

* 今回は、データセットのラベルを2値に単純化する
  * ラベル0のテキストは取り除く。
  * negativeを示す-2と-1は、一つのクラスにまとめる。
  * positiveを示す1と2も、一つのクラスにまとめる。

In [ ]:
X_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  X_binary[tag] = X[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

### 文書分類

* 適宜、チューニングしてください。
* 分類手法は`LinearSVC`でなくても構いません。

In [ ]:
from sklearn.svm import LinearSVC

cls = LinearSVC()
cls.fit(X_binary["train"], labels_binary["train"])
cls.score(X_binary["validation"], labels_binary["validation"])

## BERTによるテキストの埋め込み
* BERTにはいろいろな種類がある。
* 今日は、sentence BERTと呼ばれるBERTを使う。
* sentence BERTの説明は、今日はしない。とりあえず使う。
  * 単にテキストをembedするツールとして使う。

### 必要なライブラリのインストール

In [ ]:
!pip install -q transformers fugashi[unidic-lite]

In [ ]:
!pip install sentence-transformers

### sentence BERTのロード
* 初回だけダウンロードに時間がかかる。
* 2回目以降は、ローカルに保存したモデルをロードするだけ。

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("cl-tohoku/bert-base-japanese-v3")

### テキストの埋め込み
* 内部では、BERTの出力のmean pooling
  * 詳細は、今日のところは、割愛します。

* （おそらく2分ぐらいで終わります。）

In [ ]:
X = {}
for tag in tags:
  X[tag] = embedder.encode(texts[tag])

* embedした結果を保存しておく。

In [ ]:
import numpy as np

for tag in tags:
  with open(f'wrime_{tag}_bert_vec.npy', 'wb') as f:
    np.save(f, X[tag])

### 文書分類

In [ ]:
import numpy as np

tags = ["train", "validation", "test"]

X = {}
labels = {}
for tag in tags:
  with open(f'wrime_{tag}_bert_vec.npy', 'rb') as f:
    X[tag] = np.load(f)
  with open(f'wrime_{tag}_label.npy', 'rb') as f:
    labels[tag] = np.load(f)

* ラベルを2値に単純化する。先ほどと同じ。つまり・・・
  * ラベル0のテキストは取り除く。
  * negativeを示す-2と-1は、一つのクラスにまとめる。
  * positiveを示す1と2も、一つのクラスにまとめる。

In [ ]:
X_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  X_binary[tag] = X[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

* 適宜、チューニングしてください。
* 分類手法は`LinearSVC`でなくても構いません。

In [ ]:
from sklearn.svm import LinearSVC

cls = LinearSVC()
cls.fit(X_binary["train"], labels_binary["train"])
cls.score(X_binary["validation"], labels_binary["validation"])

# 本日の課題
* 上で実行した感情分析の性能を上げてください。
* チューニングが済んだら、テストセットでscoreを計算してください。
  * 別の評価尺度で評価してもらっても構いません。